In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import re ###?
import time
import xlsxwriter
import csv
import calendar

In [2]:
start = time.time()

#define file paths
#all traffic 
identification_tp = 'L:\\Interns2018\\Ysteinberg\\NPMRDS\\Data\\15min_data\\Truck&Passenger\\TMC_Identification.csv'
data_15_tp = 'L:\\Interns2018\\Ysteinberg\\NPMRDS\\Data\\15min_data\\Truck&Passenger\\2017-02-SPC-Region-TMCs_Multiple-Files-15min.csv'
#truck
identification_t = 'L:\\Interns2018\\Ysteinberg\\NPMRDS\\Data\\15min_data\\Truck\\TMC_Identification.csv'
data_15_t = 'L:\\Interns2018\\Ysteinberg\\NPMRDS\\Data\\15min_data\\Truck\\2017-02-SPC-Region-TMCs_Multiple-Files-15min.csv'

#read data files
identification_tp = pd.read_csv(identification_tp) #identification truck&passenger
data_15_tp = pd.read_csv(data_15_tp) #travel_time truck&passenger(all traffic)

identification_t = pd.read_csv(identification_t) #identification truck
data_15_t = pd.read_csv(data_15_t) #travel_time truck

#change column name in travel_time dataframe from tmc_code to tmc 
data_15_tp = data_15_tp.rename(columns = {'tmc_code': 'tmc'})
data_15_t = data_15_t.rename(columns = {'tmc_code': 'tmc'})

end = time.time()
print(end-start)

29.35483479499817


In [3]:
#define constant number variables
year = 2017
month = 2
directional_factor = 0.5
occupancy_factor = 1.7

interpolation = 'nearest'

#get number of days in certain month
days_month = calendar.monthrange(year,month)[1]

#to do: if Feb==28 --> 365, elif Feb==29 --> 366
days_year =365 

df1 = identification_tp
df2 = data_15_tp

In [4]:
#generate a dataframe contains data and time columns by splitting the 'measurement_tstamp' column
df_date_time = pd.DataFrame({
        'date': pd.to_datetime(df2['measurement_tstamp']).dt.date,
        'time': pd.to_datetime(df2['measurement_tstamp']).dt.time
    })

#concat dataframes of df_date_time(contains date and time columns) and df2(travel_time df) 
df22 = pd.concat([df2, df_date_time], axis=1)

#create weekday column based on 'date' column 
#In Python, lambda is just a way of defining functions inline
#for more info about lambda, please check https://www.w3schools.com/python/python_lambda.asp
df22['weekday'] = df22['date'].apply(lambda x: x.isoweekday())

#convert time column type from datetime to string as '>=' doesn't work on datetime
df22['time'] = df22['time'].astype(str)

In [7]:
columns = ['tmc', 'travel_time_seconds', 'date', 'time', 'weekday']

#travel_time df left join identification df on tmc
df_join = pd.merge(df22, df1, on='tmc', how='left')

#select interstate reporting segment data
df_inter = df_join[(df_join['f_system']==1) & (df_join['faciltype'].isin([1,2,6])) & (df_join['nhs'].isin(range(1,10))) & (df_join['urban_code']>0)]
#select non-interstate reporting segment data as LOTTR for all_traffic requires both inter and noninter state data
df_noninter = df_join[(df_join['f_system'].isin(range(2,8))) & (df_join['faciltype'].isin([1,2,6])) & (df_join['nhs'].isin(range(1,10)))& (df_join['urban_code']>0)]

#select columns for future use into dff
dff_inter = df_inter[columns]
dff_noninter = df_noninter[columns]

In [5]:
#define a function for LOTTR calculation
def LOTTR_cal(x):
#     percen_80 = stats.scoreatpercentile(x.tolist(),80) #use scipy library
#     percen_50 = stats.scoreatpercentile(x.tolist(),50)
    percen_80 = np.nanpercentile(x.tolist(), 80, interpolation = interpolation) #use numpy library
    percen_50 = np.nanpercentile(x.tolist(), 50, interpolation = interpolation)
    return percen_80/percen_50

In [ ]:
#LOTTR
dff = dff_noninter
d = {}

dff_weekday = dff[dff['weekday'].isin(range(1,6))] #monday to Friday 1-5
dff_weekend = dff[dff['weekday'].isin(range(6,8))] #Saturday to Sunday 6-7

d['LOTTR_AMP'] = dff_weekday[(dff_weekday['time']>='06:00:00') & (dff_weekday['time'] < '10:00:00')] #6am-10am-weekdays
d['LOTTR_MIDD'] = dff_weekday[(dff_weekday['time']>='10:00:00') & (dff_weekday['time'] < '16:00:00')] #10am-4pm-weekdays
d['LOTTR_PMP'] = dff_weekday[(dff_weekday['time']>='16:00:00') & (dff_weekday['time'] < '20:00:00')] #4pm-8pm-weekdays
d['LOTTR_WE'] = dff_weekend[(dff_weekend['time']>='06:00:00') & (dff_weekend['time'] < '20:00:00')] #6am-8pm-weekend

d['LOTTR_AMP'] =  LOTTR_cal(d['LOTTR_AMP']['travel_time_seconds'])
d['LOTTR_MIDD'] = LOTTR_cal(d['LOTTR_MIDD']['travel_time_seconds'])
d['LOTTR_PMP'] = LOTTR_cal(d['LOTTR_PMP']['travel_time_seconds'])
d['LOTTR_WE'] = LOTTR_cal(d['LOTTR_WE']['travel_time_seconds'])

In [10]:
d

{'LOTTR_AMP': 2.869277885235332,
 'LOTTR_MIDD': 2.893375897845172,
 'LOTTR_PMP': 2.887057010785824,
 'LOTTR_WE': 2.8830419283653033}